In [4]:
import json

# CV'leri ve ilanları yükle
with open('cv.json', 'r') as cv_file:
    cvs = json.load(cv_file)

with open('adverts.json', 'r') as advert_file:
    adverts = json.load(advert_file)

def calculate_match_score(cv, advert):
    score = 0
    # 1. Eğitim Seviyesi Eşleşmesi
    if advert["educationLevel"]["eduReq"]:
        if cv["educationInfo"]["EduLevel"] == advert["educationLevel"]["level"]:
            score += advert["educationLevel"]["eduReq"] * advert["educationLevel"]["eduReq"]
        else:
            return 0  # Gereklilik sağlanmıyor, eşleşme başarısız
    
    # 2. Deneyim Seviyesi Eşleşmesi
    if advert["experience"]["xpReq"]:
        if cv["Companies"]:
            cv_experience = sum([company["Years"] for company in cv["Companies"]])
            if cv_experience >= advert["experience"]["years"]:
                score += advert["experience"]["xpScale"] * advert["experience"]["xpReq"]
            else:
                return 0  # Gereklilik sağlanmıyor, eşleşme başarısız
        else:
            return 0  # Gereklilik sağlanmıyor
    
    # 3. Şirket Büyüklüğü Eşleşmesi
    if advert["companySize"]["compReq"]:
        if cv["Companies"]:
            company_sizes = [company["CompanySize"] for company in cv["Companies"]]
            if any(size >= advert["companySize"]["size"] for size in company_sizes):
                score += advert["companySize"]["compScale"] * advert["companySize"]["compReq"]
            else:
                return 0  # Gereklilik sağlanmıyor
    
    # 4. Beceriler Eşleşmesi
    if advert["selectedSkills"]["skillReq"]:
        matching_skills = set(cv["Skills"]).intersection(set(advert["selectedSkills"]["skills"]))
        if len(matching_skills) > 0:
            score += advert["selectedSkills"]["skillScale"] * len(matching_skills)
        else:
            return 0  # Gereklilik sağlanmıyor
    
    # 5. Diller Eşleşmesi
    if advert["selectedLanguages"]["langReq"]:
        matching_languages = set(cv["Languages"]).intersection(set(advert["selectedLanguages"]["languages"]))
        if len(matching_languages) > 0:
            score += advert["selectedLanguages"]["langScale"] * len(matching_languages)
        else:
            return 0  # Gereklilik sağlanmıyor
    
    # 6. Lokasyon Eşleşmesi
    if advert["locations"]["locReq"]:
        if set(cv["ExpectedValues"]["Locations"]).intersection(set(advert["locations"]["locations"])):
            score += advert["locations"]["locReq"] * advert["locations"]["locReq"]
        else:
            return 0  # Gereklilik sağlanmıyor
    
    return score

# Eşleşmeleri bul
matches = []

for advert in adverts:
    for cv in cvs:
        match_score = calculate_match_score(cv, advert)
        if match_score > 0:
            matches.append({
                "cvId": cv["Id"],
                "advertId": advert["Id"],
                "score": match_score
            })

# Eşleşmeleri sıralama ve yazdırma
matches = sorted(matches, key=lambda x: x['score'], reverse=True)
print(f"{len(matches)} eşleşme bulundu.")
for match in matches:
    print(f"CV {match['cvId']} ile İş İlanı {match['advertId']} uyum puanı: {match['score']}")

# İstersen eşleşmeleri bir JSON dosyasına da yazabilirsin:
with open('matches.json', 'w') as f:
    json.dump(matches, f, indent=4)

print("Eşleşmeler matches.json dosyasına kaydedildi.")


1090 eşleşme bulundu.
CV 61 ile İş İlanı 34 uyum puanı: 19
CV 8 ile İş İlanı 27 uyum puanı: 18
CV 34 ile İş İlanı 31 uyum puanı: 18
CV 43 ile İş İlanı 31 uyum puanı: 18
CV 54 ile İş İlanı 31 uyum puanı: 18
CV 61 ile İş İlanı 31 uyum puanı: 18
CV 85 ile İş İlanı 31 uyum puanı: 18
CV 93 ile İş İlanı 31 uyum puanı: 18
CV 11 ile İş İlanı 34 uyum puanı: 17
CV 16 ile İş İlanı 34 uyum puanı: 17
CV 40 ile İş İlanı 34 uyum puanı: 17
CV 51 ile İş İlanı 34 uyum puanı: 17
CV 111 ile İş İlanı 34 uyum puanı: 17
CV 119 ile İş İlanı 34 uyum puanı: 17
CV 141 ile İş İlanı 34 uyum puanı: 17
CV 47 ile İş İlanı 34 uyum puanı: 16
CV 102 ile İş İlanı 34 uyum puanı: 16
CV 5 ile İş İlanı 7 uyum puanı: 15
CV 21 ile İş İlanı 26 uyum puanı: 15
CV 24 ile İş İlanı 26 uyum puanı: 15
CV 34 ile İş İlanı 26 uyum puanı: 15
CV 35 ile İş İlanı 26 uyum puanı: 15
CV 53 ile İş İlanı 26 uyum puanı: 15
CV 54 ile İş İlanı 26 uyum puanı: 15
CV 67 ile İş İlanı 26 uyum puanı: 15
CV 68 ile İş İlanı 26 uyum puanı: 15
CV 73 ile İş İl

In [11]:
import json
import numpy as np

# Dosyaları okuma
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Dosyaya yazma
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# MSE hesaplama fonksiyonu
def calculate_mse(advert, cv):
    mse = 0
    total_weights = 0
    
    # Deneyim karşılaştırması
    if advert["experience"]["xpReq"]:
        weight = advert["experience"]["xpScale"]
        total_weights += weight
        diff = advert["experience"]["years"] - int(cv["Companies"][0]["Years"])
        mse += (diff ** 2) * weight
    
    # Şirket büyüklüğü karşılaştırması
    if advert["companySize"]["compReq"]:
        weight = advert["companySize"]["compScale"]
        total_weights += weight
        diff = advert["companySize"]["size"] - int(cv["Companies"][0]["CompanySize"])
        mse += (diff ** 2) * weight
    
    # Maaş beklentisi karşılaştırması
    if advert["salary"]["salReq"]:
        weight = advert["salary"]["salScale"]
        total_weights += weight
        diff = advert["salary"]["max"] - cv["ExpectedValues"]["ExpectedSalary"]
        mse += (diff ** 2) * weight
    
    # Eğitim seviyesi karşılaştırması
    if advert["educationLevel"]["eduReq"]:
        weight = 5  # Eğitim seviyesini sabit bir önem seviyesi ile değerlendirebiliriz
        total_weights += weight
        if advert["educationLevel"]["level"] != cv["educationInfo"]["EduLevel"]:
            mse += (1 ** 2) * weight  # Farklılık varsa 1, yoksa 0
    
    # Beceriler karşılaştırması
    if advert["selectedSkills"]["skillReq"]:
        weight = advert["selectedSkills"]["skillScale"]
        total_weights += weight
        # Ortak beceriler için farkları hesapla
        common_skills = set(advert["selectedSkills"]["skills"]).intersection(set(cv["Skills"]))
        mse += ((4 - len(common_skills)) ** 2) * weight  # Ne kadar az ortak beceri varsa, o kadar ceza
    
    # Diller karşılaştırması
    if advert["selectedLanguages"]["langReq"]:
        weight = advert["selectedLanguages"]["langScale"]
        total_weights += weight
        # Ortak diller için farkları hesapla
        common_languages = set(advert["selectedLanguages"]["languages"]).intersection(set(cv["Languages"]))
        mse += ((2 - len(common_languages)) ** 2) * weight  # Ne kadar az ortak dil varsa, o kadar ceza
    
    # Şehir karşılaştırması
    if advert["locations"]["locReq"]:
        weight = 5  # Şehirler için sabit bir önem seviyesi kullanılabilir
        total_weights += weight
        if advert["locations"]["locations"][0] != cv["ExpectedValues"]["Locations"][0]:
            mse += (1 ** 2) * weight  # Farklılık varsa 1, yoksa 0
    
    # MSE'yi toplam ağırlıklara bölerek normalize edelim
    if total_weights > 0:
        mse /= total_weights
    
    return mse

# İş ilanları ve CV'ler için eşleştirme
def match_adverts_to_cvs(advert_list, cv_list):
    matches = []
    
    for advert in advert_list:
        scores = []
        for cv in cv_list:
            score = calculate_mse(advert, cv)
            scores.append((cv["Id"], score))
        
        # En düşük skora sahip CV'leri alalım
        scores.sort(key=lambda x: x[1])
          # İlk 3 en iyi eşleşme
        
        matches.append({
            "AdvertId": advert["Id"],
            "TopMatches": scores
        })
    
    return matches

# Ana program akışı
def main():
    # JSON dosyalarını yükle
    adverts = load_json('adverts.json')
    cvs = load_json('cv.json')
    
    # Eşleşmeleri hesapla
    matched_cvs = match_adverts_to_cvs(adverts, cvs)
    
    # Sonuçları matched_cv.json dosyasına kaydet
    save_json(matched_cvs, 'matched_cv.json')
    
    print("Eşleşmeler başarıyla kaydedildi.")

# Programı çalıştır
if __name__ == "__main__":
    main()


Eşleşmeler başarıyla kaydedildi.


In [13]:
import json
import numpy as np

# Dosyaları okuma
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Dosyaya yazma
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# Normalized MSE hesaplama ve başarı oranı elde etme fonksiyonu
def calculate_success_rate(advert, cv):
    mse = 0
    total_weights = 0
    
    # Deneyim karşılaştırması
    if advert["experience"]["xpReq"]:
        weight = advert["experience"]["xpScale"]
        total_weights += weight
        diff = advert["experience"]["years"] - int(cv["Companies"][0]["Years"])
        mse += (diff ** 2) * weight
    
    # Şirket büyüklüğü karşılaştırması
    if advert["companySize"]["compReq"]:
        weight = advert["companySize"]["compScale"]
        total_weights += weight
        diff = advert["companySize"]["size"] - int(cv["Companies"][0]["CompanySize"])
        mse += (diff ** 2) * weight
    
    # Maaş beklentisi karşılaştırması
    if advert["salary"]["salReq"]:
        weight = advert["salary"]["salScale"]
        total_weights += weight
        diff = advert["salary"]["max"] - cv["ExpectedValues"]["ExpectedSalary"]
        mse += (diff ** 2) * weight
    
    # Eğitim seviyesi karşılaştırması
    if advert["educationLevel"]["eduReq"]:
        weight = 5  # Eğitim seviyesini sabit bir önem seviyesi ile değerlendirebiliriz
        total_weights += weight
        if advert["educationLevel"]["level"] != cv["educationInfo"]["EduLevel"]:
            mse += (1 ** 2) * weight  # Farklılık varsa 1, yoksa 0
    
    # Beceriler karşılaştırması
    if advert["selectedSkills"]["skillReq"]:
        weight = advert["selectedSkills"]["skillScale"]
        total_weights += weight
        # Ortak beceriler için farkları hesapla
        common_skills = set(advert["selectedSkills"]["skills"]).intersection(set(cv["Skills"]))
        mse += ((4 - len(common_skills)) ** 2) * weight  # Ne kadar az ortak beceri varsa, o kadar ceza
    
    # Diller karşılaştırması
    if advert["selectedLanguages"]["langReq"]:
        weight = advert["selectedLanguages"]["langScale"]
        total_weights += weight
        # Ortak diller için farkları hesapla
        common_languages = set(advert["selectedLanguages"]["languages"]).intersection(set(cv["Languages"]))
        mse += ((2 - len(common_languages)) ** 2) * weight  # Ne kadar az ortak dil varsa, o kadar ceza
    
    # Şehir karşılaştırması
    if advert["locations"]["locReq"]:
        weight = 1  # Şehirler için sabit bir önem seviyesi kullanılabilir
        total_weights += weight
        if advert["locations"]["locations"][0] != cv["ExpectedValues"]["Locations"][0]:
            mse += (1 ** 2) * weight  # Farklılık varsa 1, yoksa 0
    
    # MSE'yi toplam ağırlıklara bölerek normalize edelim
    if total_weights > 0:
        mse /= total_weights
    
    # MSE'yi başarı oranına dönüştürmek için 1'den çıkartıyoruz (başarı oranı 0-1 arası)
    success_rate = 1 - mse
    return success_rate

# İş ilanları ve CV'ler için eşleştirme
def match_adverts_to_cvs(advert_list, cv_list, threshold=0.7):
    matches = []
    
    for advert in advert_list:
        scores = []
        for cv in cv_list:
            success_rate = calculate_success_rate(advert, cv)
            if success_rate >= threshold:  # Eşik üzerinde olan başarı oranlarını kabul et
                scores.append((cv["Id"], success_rate))
        
        # Başarı oranına göre en iyi eşleşmeleri sıralıyoruz
        scores.sort(key=lambda x: x[1], reverse=True)
        
        # Eşleşme bulunan CV'leri kaydediyoruz
        if scores:
            matches.append({
                "AdvertId": advert["Id"],
                "TopMatches": scores  # İlk 3 en iyi eşleşme
            })
    
    return matches

# Ana program akışı
def main():
    # JSON dosyalarını yükle
    adverts = load_json('adverts.json')
    cvs = load_json('cv.json')
    
    # Eşleşmeleri hesapla, threshold 0.7 olarak belirlendi
    matched_cvs = match_adverts_to_cvs(adverts, cvs, threshold=0.7)
    
    # Sonuçları matched_cv.json dosyasına kaydet
    save_json(matched_cvs, 'matched.json')
    
    print("Eşleşmeler başarıyla kaydedildi.")

# Programı çalıştır
if __name__ == "__main__":
    main()


Eşleşmeler başarıyla kaydedildi.


In [51]:
import json

def calculate_success_rate(advert, cv):
    mse = 0
    total_weights = 0

    # Deneyim 
    weight = advert["experience"]["xpScale"]
    total_weights += weight
    total_experience_years = sum(int(company["Years"]) for company in cv["Companies"])
    if advert["experience"]["xpReq"] and total_experience_years >= advert["experience"]["years"]:
        mse +=0
    elif (not advert["experience"]["xpReq"]):
        if total_experience_years >= advert["experience"]["years"]:
            mse+=0
        else:
            diff=advert["experience"]["years"]-total_experience_years
            if advert["experience"]["years"]==0:
                err=diff/0.1

            else:
                err=diff/advert["experience"]["years"]
                
            mse += (err ** 2) * weight

    else:
        mse+=10000000
        


    # Şirket büyüklüğü karşılaştırması (en büyük şirkete bakıyoruz)
    weight = advert["companySize"]["compScale"]
    total_weights += weight
    largest_company_size = max(int(company["CompanySize"]) for company in cv["Companies"])
    if advert["companySize"]["compReq"] and largest_company_size >= advert["companySize"]["size"]:
        mse+=0
    elif (not advert["companySize"]["compReq"]):
        if largest_company_size >= advert["companySize"]["size"]:
            mse+=0
        else:
            if advert["companySize"]["size"]<=largest_company_size:
                mse+=0
            else:
                diff = advert["companySize"]["size"] - largest_company_size
                if advert["companySize"]["size"]<=0:
                    err=diff/0.1
                else:
                    err=diff/advert["companySize"]["size"]
                mse += (err ** 2) * weight
            
    else:
        mse+=10000000
        

        
        
            
    
    # Maaş beklentisi karşılaştırması

    weight = advert["salary"]["salScale"]
    total_weights += weight
    if advert["salary"]["salReq"] and cv["ExpectedValues"]["ExpectedSalary"] >= advert["salary"]["max"]:
        mse+=0
    elif( not advert["salary"]["salReq"]) :
        if cv["ExpectedValues"]["ExpectedSalary"] >= advert["salary"]["max"]:
            mse+=0
        else:
            diff = cv["ExpectedValues"]["ExpectedSalary"] - advert["salary"]["max"]
            if advert["salary"]["max"]<=0:
                err=diff/0.1
            else:
                err=diff/advert["salary"]["max"]
            mse += (err ** 2) * weight
    else:
        mse+=10000000
        
        
            
    

    # Beceriler karşılaştırması
    required_skills = set(advert["selectedSkills"]["skills"])
    candidate_skills = set(cv["Skills"])
    common_skills = required_skills.intersection(candidate_skills)
    
        
    weight = advert["selectedSkills"]["skillScale"]
    total_weights += weight
        
    
    if advert["selectedSkills"]["skillReq"] and common_skills==required_skills:
        mse+=0
    elif (not advert["selectedSkills"]["skillReq"]) :
        if common_skills==required_skills:
            mse+=0
        else:
            if len(required_skills)<=0:
                err=1-(len(common_skills)/0.1)
            else:
                err=1-(len(common_skills)/len(required_skills))
            mse += (err**2) * weight
    else:
        mse+=10000000
        


    # Diller karşılaştırması
    required_languages=set(advert["selectedLanguages"]["languages"])
    candidate_languages = set(cv["Languages"])
    common_languages = required_languages.intersection(candidate_languages)
    weight = advert["selectedLanguages"]["langScale"]
    total_weights += weight
    if advert["selectedLanguages"]["langReq"] and common_languages==required_languages:
        mse+=0
    elif (not advert["selectedLanguages"]["langReq"] ):
        if common_languages==required_languages:
            mse+=0
        else:
            if len(required_languages)<=0:
                err=1
            else:
                err=1-len(common_languages)/len(required_languages)
            mse += (err**2) * weight
    else:
        mse+=10000000
        
    

    # Şehir karşılaştırması
    required_cities=set(advert["locations"]["locations"])
    candidate_cities=set(cv["ExpectedValues"]["Locations"])
    common_cities=required_cities.intersection(candidate_cities)

    if len(common_cities)<=0:
        mse+=100000000
    else:
        mse+=0
    
    # MSE'yi normalize edip başarı oranına dönüştür
    if total_weights > 0:
        mse /= total_weights
    
    success_rate = 1 - mse
    success_rate = round(success_rate, 3)
    return success_rate

def match_cvs_with_adverts():
    threshold=0.6
    with open('cv.json', 'r', encoding='utf-8') as f:
        cvs = json.load(f)
    
    with open('adverts.json', 'r', encoding='utf-8') as f:
        adverts = json.load(f)
    
    matched_cvs = []
    
    
    for advert in adverts:
        for cv in cvs:
            success_rate = calculate_success_rate(advert, cv)
            if success_rate >= threshold:
                matched_cvs.append({
                    "advertID": advert["Id"],
                    "cvId": cv["Id"],
                    "score": success_rate
                })

    with open("matched_cvs.json", 'w', encoding='utf-8') as f:
        json.dump(matched_cvs, f, ensure_ascii=False, indent=4)

def match_adverts_with_cvs():
    threshold=0.6
    with open('cv.json', 'r', encoding='utf-8') as f:
        cvs = json.load(f)
    
    with open('adverts.json', 'r', encoding='utf-8') as f:
        adverts = json.load(f)
    
    matched_adverts = []
    
    
    for cv in cvs:
        for advert in adverts:
            success_rate = calculate_success_rate(advert, cv)
            if success_rate >= threshold:
                matched_adverts.append({
                    "cvId": cv["Id"],
                    "advertID": advert["Id"],
                    "score": success_rate  
                })

    with open("matched_adverts.json", 'w', encoding='utf-8') as f:
        json.dump(matched_adverts, f, ensure_ascii=False, indent=4)



match_cvs_with_adverts()
match_adverts_with_cvs()


In [ ]:
import json
import requests
import os
from flask import Flask, request, jsonify

app = Flask(__name__)

# Eşleşme oranını hesaplayan fonksiyon
def calculate_success_rate(advert, cv):
    return 0.7  # Örnek başarı oranı

# API'den veri çekme fonksiyonu
def fetch_data_from_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API isteği başarısız oldu: {response.status_code}")

# CV ve ilanları eşleştirip matched_cvs.json dosyasına yazma
def match_cvs_with_adverts():
    threshold = 0.6

    # .NET API'den verileri çek
    cvs = fetch_data_from_api('https://localhost:7190/cv')
    adverts = fetch_data_from_api('https://localhost:7190/advert')

    matched_cvs = []

    for advert in adverts:
        for cv in cvs:
            success_rate = calculate_success_rate(advert, cv)
            if success_rate >= threshold:
                matched_cvs.append({
                    "advertID": advert["Id"],
                    "cvId": cv["Id"],
                    "score": success_rate
                })

    # matched_cvs.json dosyasına yazma
    output_dir = r'C:\Users\Furkan\Documents\kariyer-app-data'
    output_path = os.path.join(output_dir, 'matched_cvs.json')
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(matched_cvs, f, ensure_ascii=False, indent=4)

# İlanları CV'lerle eşleştirip matched_adverts.json dosyasına yazma
def match_adverts_with_cvs():
    threshold = 0.6

    # .NET API'den verileri çek
    cvs = fetch_data_from_api('https://localhost:7190/cv')
    adverts = fetch_data_from_api('https://localhost:7190/advert')

    matched_adverts = []

    for cv in cvs:
        for advert in adverts:
            success_rate = calculate_success_rate(advert, cv)
            if success_rate >= threshold:
                matched_adverts.append({
                    "cvId": cv["Id"],
                    "advertID": advert["Id"],
                    "score": success_rate  
                })

    # matched_adverts.json dosyasına yazma
    output_dir = r'C:\Users\Furkan\Documents\kariyer-app-data'
    output_path = os.path.join(output_dir, 'matched_adverts.json')

    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(matched_adverts, f, ensure_ascii=False, indent=4)

# Vue.js'den gelen POST isteğini dinleyen Flask route'u
@app.route('/api/trigger', methods=['POST'])
def handle_trigger():
    try:
        # Vue.js'den uyarı geldi, eşleştirme işlemini başlatıyoruz
        match_cvs_with_adverts()
        match_adverts_with_cvs()

        # İşlem tamamlandıktan sonra .NET API'ye sinyal gönderiyoruz
        notify_dotnet_api()

        return jsonify({"message": "Eşleştirme işlemi tamamlandı ve .NET API'ye sinyal gönderildi."}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# .NET API'ye eşleştirme işlemi sonrası sinyal gönderme
def notify_dotnet_api():
    try:
        # .NET API'ye POST isteği gönderme
        requests.post('https://localhost:7190/api/update')
        print(".NET API'ye başarıyla sinyal gönderildi.")
    except Exception as e:
        print(f".NET API'ye sinyal gönderme başarısız oldu: {str(e)}")

if __name__ == '__main__':
    app.run(port=5000, debug=True)
